# Analyzing borrowers’ risk of defaulting

The aim of this project is to prepare a report for a bank’s loan division. The main thing that I'm showing is if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

This report will be considered when building a **credit scoring** of a potential customer. A ** credit scoring ** is used to evaluate the ability of a potential borrower to repay their loan.

## Open the data file and have a look at the general information. 

In [1]:
import pandas as pd

try:
    df=pd.read_csv('credit_scoring_eng.csv')
except:
    df=pd.read_csv('/datasets/credit_scoring_eng.csv')

df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


### Conclusion

I found some issues with the report's data which I'll address them below. Those are the issues that I've found

1. There are duplicate rows in the date source.
2. column children, some has a negative value. I believe that this is a typo, and I removed the '-' sign.
3. Days Employed column, has negative values, the value is in days, rather than in years, has missing values, 
    and some have very large valuse
4. Missing values in the total income column, and making it more readable by changing the value's type to integer.

## Data preprocessing

### Processing duplicates

To eliminate the chance of missing a duplicate for a different casing, 
I'll make sure that each column has the same case

In [2]:
df['education']=df['education'].astype(str).str.lower()
df['family_status']=df['family_status'].astype(str).str.lower()
df['gender']=df['gender'].astype(str).str.upper()
df['income_type']=df['income_type'].astype(str).str.lower()
df['purpose']=df['purpose'].astype(str).str.lower()

In [3]:
#check if there are duplicates in the datasource
num_of_duplicate=df.duplicated().sum()
pct_of_duplicate=num_of_duplicate/len(df)
print('Number of duplicates in the dataset: {}, which is {:.2%} of the dataset'
      .format(num_of_duplicate,pct_of_duplicate))

Number of duplicates in the dataset: 71, which is 0.33% of the dataset


In [4]:
#since I found that there are 71 duplicates, i'm erasing them.
df.drop_duplicates(inplace=True)
print('Num of remaining duplicates in the dataset:',df.duplicated().sum())

Num of remaining duplicates in the dataset: 0


### Conclusion

Although the duplicate data may not have an impact on the end result, it is a type of error, and it must be handled.
A possible reason for a duplicate is a wrong SQL query, which should be fixed by the developer.

### Processing missing values

In [5]:
#days_employed column
days_employed_mis_val_num=df['days_employed'].isnull().sum()
print('Num of missing values at column days_employed:',days_employed_mis_val_num)
days_employed_mis_val_pct=days_employed_mis_val_num/len(df)
print('Percent of missing values at column days_employed: {:.2%}'.format(days_employed_mis_val_pct))

Num of missing values at column days_employed: 2103
Percent of missing values at column days_employed: 9.80%


In [6]:
#filling missing values with 0
try:
    df['days_employed']=df['days_employed'].fillna(0)
except Exception as e: 
    print(e)
    
#I've noticed that there are negative values. 
#Since they'll might have an impact on the median value, I have to find
#the median after treating them. The source for the negative values is
#probably a typo
df['days_employed']=abs(df['days_employed'])

#finding the median value for years of employment
median_employment_years=round(df['days_employed'].median())
#replcaing 0 years of employment with the median value that I've found
df.loc[df['days_employed']==0, 'days_employed']=median_employment_years

In [7]:
#handling total income's missing values
total_income_mis_val_num=df['total_income'].isnull().sum()
print('Num of missing values at column total_income:',total_income_mis_val_num)
total_income_mis_val_pct=total_income_mis_val_num/len(df)
print('Percent of missing values at column total_income: {:.2%}'.format(total_income_mis_val_pct))

Num of missing values at column total_income: 2103
Percent of missing values at column total_income: 9.80%


In [8]:
#i believe that the value is missing, and not that the person has 0 income, 
#or else that person wouldn't take a loan.
total_income_madian=round(df['total_income'].median())
df.loc[df['total_income'].isnull(),'total_income']=total_income_madian

### Conclusion

2 places are having missing values, on columns:
    days_employed, total_income
To have minimum impact on the calculation of probabilities, I used the median value as a filler to those empty holes.
Reasons for missing values can vary from missing information, problems on an ETL.

### Data type replacement

In 2 columns I've changed the data type so that the report will be more readable.
I made the change on:
    total_income, days_employed

In [9]:
#I transfered the days_employed to years by deviding in 365, 
#then I changed the type to integer
df['days_employed'] = (df['days_employed']/365).astype('int')
#then i renamed the column name to years_employed
new_column_names=['children', 'years_employed', 'dob_years', 'education', 'education_id', 'family_status', 'family_status_id', 'gender', 'income_type', 'debt', 'total_income', 'purpose']
df.set_axis(new_column_names, axis='columns', inplace=True)

In [10]:
#the total_income type change to integer
df['total_income']=df['total_income'].astype('int')

### Conclusion

The main reason for data change is to make the report more readable, and although there is i slight data loss, it has no impact on the final report's outcome

### Bad values treatment

After getting the year_employed I've noticed that there are values in years employed of over 1000 years. I decided that more than 60 years is not logical, and replaced it with the median value

In [11]:
print(df.loc[df['years_employed']>60]['years_employed'].head())

4      932
18    1096
24     927
25     996
30     919
Name: years_employed, dtype: int64


In [12]:
#replace value too big for years of employment with 0
df.loc[df['years_employed']>60, 'years_employed']=0
df.loc[df['years_employed']==0, 'years_employed']=median_employment_years

Looking at the number of children I've noticed that there are 76 loaners with 20 kids. I believe that it's a typo, and should have been 2. Also, there are 47 loaners with -1 children, which probably should be 1

In [13]:
print(df['children'].value_counts())

 0     14091
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


In [14]:
df.loc[df['children']==20,'children']=2
df['children']=df['children'].abs()
print(df['children'].value_counts())

0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64


### Conclusion

After identifying that some of the values are not reasonable, I replaced them with the median value so that the statistical calculation won't be affected.

### Categorizing Data

To categories data, I used the stemmer. First of all, I needed to see how each word comes out of the stemmer.

In [15]:
from nltk.stem import SnowballStemmer

english_stemmer = SnowballStemmer('english')

print("english_stemmer.stem('house')",english_stemmer.stem('house'))
print("english_stemmer.stem('education')",english_stemmer.stem('education'))
print("english_stemmer.stem('wedding')",english_stemmer.stem('wedding'))
print("english_stemmer.stem('estate')",english_stemmer.stem('estate'))
print("english_stemmer.stem('property')",english_stemmer.stem('property'))
print("english_stemmer.stem('university')",english_stemmer.stem('university'))
print("english_stemmer.stem('estate')",english_stemmer.stem('estate'))

english_stemmer.stem('house') hous
english_stemmer.stem('education') educ
english_stemmer.stem('wedding') wed
english_stemmer.stem('estate') estat
english_stemmer.stem('property') properti
english_stemmer.stem('university') univers
english_stemmer.stem('estate') estat


According to the words that come out of the stemmer, I made a function to categorize the reasons for a loan.
I ran it on column 'purpose'. I kept the new categories in a new column called 'purpose_category'

In [16]:
def categorize_purpose(purpose):
    words=purpose.split(' ')
    for word in words:
        stemmed_word=english_stemmer.stem(word)
        if stemmed_word == 'car':
            return 'car'
        elif stemmed_word in ['educ','univers']:
            return 'education'
        elif stemmed_word in ['hous','estat','properti']:
            return 'house'
        elif stemmed_word =='wed':
            return 'wedding'
    return ''

df['purpose_category']=df['purpose'].apply(categorize_purpose)

#checking that all purposes are categoriesed
print(df[df['purpose_category']==''])

Empty DataFrame
Columns: [children, years_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose, purpose_category]
Index: []


In [17]:
#the categories that I came up with are
print(df['purpose_category'].value_counts())

house        10811
car           4306
education     4013
wedding       2324
Name: purpose_category, dtype: int64


### Conclusion

The most common reason for taking a loan is for housing, its size is like the other 3 categories combined.

## Answer these questions

- Is there a relation between having kids and repaying a loan on time?

In [18]:
df_pivot = df.pivot_table(
    index='children', values='debt', aggfunc=['sum','count'])
df_pivot['percentage']=(df_pivot['sum']['debt']/df_pivot['count']['debt']*100)
df_pivot

,sum,count,percentage
,debt,debt,
children,,,
0,1063,14091,7.543822
1,445,4855,9.165808
2,202,2128,9.492481
3,27,330,8.181818
4,4,41,9.756098
5,0,9,0.000000


### Conclusion

After looking on the percent of defaulting loaners per child, I can see that the default rate is nearly the same for loaners with children, no metter how many children there are. On the other hand, loaners without kids has higher return rate.

- Is there a relation between marital status and repaying a loan on time?

In [19]:
df_pivot = df.pivot_table(
    index='family_status', values='debt', aggfunc=['sum','count'])

df_pivot['percentage']=(df_pivot['sum']['debt']/df_pivot['count']['debt']*100)
df_pivot

,sum,count,percentage
,debt,debt,
family_status,,,
civil partnership,388,4151,9.347145
divorced,85,1195,7.112971
married,931,12339,7.545182
unmarried,274,2810,9.750890
widow / widower,63,959,6.569343


### Conclusion

Those who made a commitment, and got married, had a lower loan defaulting rate

- Is there a relation between income level and repaying a loan on time?

To achieve that goal, I'll categorize the income levels into 5 levels: low, below average, average, above average, high

In [20]:
#checking the mean & median values for total income column, according to it, I'll make the total income groups
print('The total income mean value is:',df['total_income'].mean())
print('The total income median value is:',df['total_income'].median())

The total income mean value is: 26435.747413069825
The total income median value is: 23203.0


In [21]:
df.sort_values(by=['total_income'], ascending=False)

def categorize_total_income(total_income):
    if total_income < 10000:
        return 'low'
    elif total_income < 20000:
        return 'below average'
    elif total_income < 30000:
        return 'average'
    elif total_income < 50000:
        return 'above average'
    else:
        return 'high'

df['total_income_category']=df['total_income'].apply(categorize_total_income)

#checking how many loaners are in each group
df['total_income_category'].value_counts()

average          8166
below average    6443
above average    4599
high             1320
low               926
Name: total_income_category, dtype: int64

In [22]:
df_pivot = df.pivot_table(
    index='total_income_category', values='debt', aggfunc=['sum','count'])

df_pivot['percentage']=(df_pivot['sum']['debt']/df_pivot['count']['debt']*100)
df_pivot

,sum,count,percentage
,debt,debt,
total_income_category,,,
above average,344,4599,7.479887
average,697,8166,8.535391
below average,550,6443,8.536396
high,92,1320,6.969697
low,58,926,6.263499


### Conclusion

Most people are in the average and below-average categories, and they have a higher default rate. The low-income group, which seems to be the most dangerous, has the lowest defaulting rates

- How do different loan purposes affect on-time repayment of the loan?

In [31]:
df_pivot=df.pivot_table(index='purpose_category',values='debt', aggfunc=['sum','count'])
df_pivot['default_rate']=(df_pivot['sum']['debt']/df_pivot['count']['debt']*100)
df_pivot

,sum,count,default_rate
,debt,debt,
purpose_category,,,
car,403,4306,9.359034
education,370,4013,9.220035
house,782,10811,7.233373
wedding,186,2324,8.003442


### Conclusion

Grouping by the purpose category shows that loans for housing are the safest, riskier are loans for weddings, and the loans with the highest risk are loans for buying cars or education.

## General conclusion

Although I got data that is not that neat and clean, I managed to draw conclusions out of it. To save time and get better results, it is advised to ask the developers to fix those issues

## Project Readiness Checklist

Put 'x' in the completed points. Then press Shift + Enter.
- [x]  file open;
- [x]  file examined;
- [x]  missing values defined;
- [x]  missing values are filled;
- [x]  an explanation of which missing value types were detected;
- [x]  explanation for the possible causes of missing values;
- [x]  an explanation of how the blanks are filled;
- [x]  replaced the real data type with an integer;
- [x]  an explanation of which method is used to change the data type and why;
- [x]  duplicates deleted;
- [x]  an explanation of which method is used to find and remove duplicates;
- [x]  description of the possible reasons for the appearance of duplicates in the data;
- [x]  data is categorized;
- [x]  an explanation of the principle of data categorization;
- [x]  an answer to the question "Is there a relation between having kids and repaying a loan on time?";
- [x]  an answer to the question " Is there a relation between marital status and repaying a loan on time?";
- [x]   an answer to the question " Is there a relation between income level and repaying a loan on time?";
- [x]  an answer to the question " How do different loan purposes affect on-time repayment of the loan?"
- [x]  conclusions are present on each stage;
- [x]  a general conclusion is made.